## Chicago Crime Data analysis

This dataset reflects reported incidents of crime (with the exception of murders where data exists for each victim) that occurred in the City of Chicago from 2001 to present.
Using this CSV file we're solving few problem statments using spark scala & visualizing them through graph & table view.

Data source  - https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-Present/ijzp-q8t2

In [0]:
%scala
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions.{col, to_date}

// To define user defined schema and specify datatype
val schema = StructType(Array(
    StructField("ID",IntegerType,true),
    StructField("Case Number",StringType,true),
    StructField("Date",StringType,true),
    StructField("Block",StringType,true),
    StructField("IUCR", StringType,true),
    StructField("Primary Type",StringType,true),
    StructField("Description",StringType,true),
    StructField("Location Description",StringType,true),
    StructField("Arrest",BooleanType,true),
    StructField("Domestic", BooleanType,true),
    StructField("Beat",IntegerType,true),
    StructField("District",IntegerType,true),
    StructField("Ward",IntegerType,true),
    StructField("Community Area",IntegerType,true),
    StructField("FBI Code",StringType,true),
    StructField("X Coordinate",IntegerType,true),
    StructField("Y Coordinate",IntegerType,true),
    StructField("Year",IntegerType,true),
    StructField("Updated On",StringType,true),
    StructField("Latitude",DoubleType,true),
    StructField("Longitude",DoubleType,true),
    StructField("Location",StringType,true)
  ))

val crimeDF = spark.read.format("com.databricks.spark.csv").option("header",true).schema(schema).csv("/FileStore/tables/Crimes___2001_to_Present.csv")
crimeDF.printSchema()
crimeDF.show(false)

root
-- ID: integer (nullable = true)
-- Case Number: string (nullable = true)
-- Date: string (nullable = true)
-- Block: string (nullable = true)
-- IUCR: string (nullable = true)
-- Primary Type: string (nullable = true)
-- Description: string (nullable = true)
-- Location Description: string (nullable = true)
-- Arrest: boolean (nullable = true)
-- Domestic: boolean (nullable = true)
-- Beat: integer (nullable = true)
-- District: integer (nullable = true)
-- Ward: integer (nullable = true)
-- Community Area: integer (nullable = true)
-- FBI Code: string (nullable = true)
-- X Coordinate: integer (nullable = true)
-- Y Coordinate: integer (nullable = true)
-- Year: integer (nullable = true)
-- Updated On: string (nullable = true)
-- Latitude: double (nullable = true)
-- Longitude: double (nullable = true)
-- Location: string (nullable = true)

+--------+-----------+----------------+-----------------------+----+------------------+----------------------------------------------+------------------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+----------------------+-----------+------------+-----------------------------+
ID |Case Number|Date |Block |IUCR|Primary Type |Description |Location Description |Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|Updated On |Latitude |Longitude |Location |
+--------+-----------+----------------+-----------------------+----+------------------+----------------------------------------------+------------------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+----------------------+-----------+------------+-----------------------------+
10224738|HY411648 |09-05-2015 13:30|043XX S WOOD ST |486 |BATTERY |DOMESTIC BATTERY SIMPLE |RESIDENCE |false |true |924 |9 |12 |61 |08B |1165074 |1875917 |2015|02-10-2018 15:50 |41.81511728|-87.66999956|(41.815117282, -87.669999562)|
10224739|HY411615 |09-04-2015 11:30|008XX N CENTRAL AVE |870 |THEFT |POCKET-PICKING |CTA BUS |false |false |1511|15 |29 |25 |6 |1138875 |1904869 |2015|02-10-2018 15:50 |41.89508047|-87.76540045|(41.895080471, -87.765400451)|
11646166|JC213529 |09-01-2018 00:01|082XX S INGLESIDE AVE |810 |THEFT |OVER $500 |RESIDENCE |false |true |631 |6 |8 |44 |6 |null |null |2018|04-06-2019 16:04 |null |null |null |
10224740|HY411595 |09-05-2015 12:45|035XX W BARRY AVE |2023|NARCOTICS |POSS: HEROIN(BRN/TAN) |SIDEWALK |true |false |1412|14 |35 |21 |18 |1152037 |1920384 |2015|02-10-2018 15:50 |41.93740577|-87.71664969|(41.937405765, -87.716649687)|
10224741|HY411610 |09-05-2015 13:00|0000X N LARAMIE AVE |560 |ASSAULT |SIMPLE |APARTMENT |false |true |1522|15 |28 |25 |08A |1141706 |1900086 |2015|02-10-2018 15:50 |41.88190344|-87.75512115|(41.881903443, -87.755121152)|
10224742|HY411435 |09-05-2015 10:55|082XX S LOOMIS BLVD |610 |BURGLARY |FORCIBLE ENTRY |RESIDENCE |false |false |614 |6 |21 |71 |5 |1168430 |1850165 |2015|02-10-2018 15:50 |41.74437888|-87.65843064|(41.744378879, -87.658430635)|
10224743|HY411629 |09-04-2015 18:00|021XX W CHURCHILL ST |620 |BURGLARY |UNLAWFUL ENTRY |RESIDENCE-GARAGE |false |false |1434|14 |32 |24 |5 |1161628 |1912157 |2015|02-10-2018 15:50 |41.9146356 |-87.68163091|(41.914635603, -87.681630909)|
10224744|HY411605 |09-05-2015 13:00|025XX W CERMAK RD |860 |THEFT |RETAIL THEFT |GROCERY FOOD STORE |true |false |1034|10 |25 |31 |6 |1159734 |1889313 |2015|09/17/2015 11:37:18 AM|41.85198889|-87.68921912|(41.851988885, -87.689219118)|
10224745|HY411654 |09-05-2015 11:30|031XX W WASHINGTON BLVD|320 |ROBBERY |STRONGARM - NO WEAPON |STREET |false |true |1222|12 |27 |27 |3 |1155536 |1900515 |2015|02-10-2018 15:50 |41.88281374|-87.70432572|(41.88281374, -87.704325717) |
11645836|JC212333 |05-01-2016 00:25|055XX S ROCKWELL ST |1153|DECEPTIVE PRACTICE|FINANCIAL IDENTITY THEFT OVER $ 300 |null |false |false |824 |8 |15 |63 |11 |null |null |2016|04-06-2019 16:04 |null |null |null |
10224746|HY

In [0]:
%scala
//   Create a view or table

crimeDF.createOrReplaceTempView("crime_table")

In [0]:
%scala

display(spark.sql("SELECT `Primary Type`,Year, COUNT(*) AS `Crime Count` FROM crime_table GROUP BY Year,`Primary Type` ORDER BY Year, `Primary Type` DESC"))


Primary Type,Year,Crime Count
WEAPONS VIOLATION,2001,243
THEFT,2001,5250
STALKING,2001,19
SEX OFFENSE,2001,101
ROBBERY,2001,1086
RITUALISM,2001,1
PUBLIC PEACE VIOLATION,2001,122
PROSTITUTION,2001,400
OTHER OFFENSE,2001,1953
OTHER NARCOTIC VIOLATION,2001,1


Output can only be rendered in Databricks

In [0]:
%scala
display(spark.sql("select Year,count(*) from crime_table where Arrest='true' group by Year order by Year"))


Year,count(1)
2001,8730
2002,222
2003,1
2004,2
2005,4
2006,3
2007,2
2008,4
2009,14
2010,15


Output can only be rendered in Databricks

In [0]:
%scala

//  Step 1: Extracting months from Date column and grouping the crimes per month
val month_count=spark.sql("select substring(Date,0,2) as Month_val,count(*) as noofcrimes from crime_table group by Month_val order by Month_val")

// Step 2: Using lag function to find variation in crime compared with months
month_count.createOrReplaceTempView("month_counts")
val finalresult7=spark.sql("""select *,abs(previouscount-noofcrimes) as variation_value from(select *,LAG(noofcrimes,1) over (order by Month_val) as previouscount from month_counts) where previouscount is not null""")

// Step 3: Displaying month having highest variation in crime
finalresult7.createOrReplaceTempView("result")
spark.sql("select Month_val as Month_with_highest_variation from result where variation_value=(select max(variation_value) as high_count from result)").show()


+----------------------------+
Month_with_highest_variation|
+----------------------------+
 02|
+----------------------------+

month_count: org.apache.spark.sql.DataFrame = [Month_val: string, noofcrimes: bigint]
finalresult7: org.apache.spark.sql.DataFrame = [Month_val: string, noofcrimes: bigint ... 2 more fields]

In [0]:

%scala
import org.apache.spark.sql.functions.{col, to_date}
// Step 1: Extracting date in string format and count of crimes for all the dates
val datedf=spark.sql("SELECT substring(Date,0,10) as Date_val,count(ID) as crimeCountOfDate  from crime_table group by Date_val")

// Step 2: converting dates from string to date format
val newDate=datedf.select(col("Date_val"),
    to_date(col("Date_val"),"MM/dd/yyyy").as("finalDate"),col("crimeCountOfDate"))

// Step 3: Grouping by quarters and finding the sum of crime counts per quarter
newDate.createOrReplaceTempView("newDate")
val resultdf=spark.sql("select quarter(finalDate) as quarters,sum(crimeCountOfDate) as totalCrimeCountPerQuarter from newDate group by quarters order by quarters")

// Step 4: using lag function to compare the variation of crime counts with the previous quarter
resultdf.createOrReplaceTempView("resultdf")
val result=spark.sql("""
select *,abs(previous_count-totalCrimeCountPerQuarter)as count_varition_per_quarter from (
select *,LAG(totalCrimeCountPerQuarter) OVER (ORDER BY quarters) as previous_count from resultdf)as a where previous_count is not null""")

// Step 5: finding the peaceful quarter compared to previous quarter
result.createOrReplaceTempView("crime_result")
spark.sql("select quarters from crime_result where count_varition_per_quarter=(select min(count_varition_per_quarter) from crime_result)").show


+--------+
quarters|
+--------+
 4|
+--------+

import org.apache.spark.sql.functions.{col, to_date}
datedf: org.apache.spark.sql.DataFrame = [Date_val: string, crimeCountOfDate: bigint]
newDate: org.apache.spark.sql.DataFrame = [Date_val: string, finalDate: date ... 1 more field]
resultdf: org.apache.spark.sql.DataFrame = [quarters: int, totalCrimeCountPerQuarter: bigint]
result: org.apache.spark.sql.DataFrame = [quarters: int, totalCrimeCountPerQuarter: bigint ... 2 more fields]

In [0]:
%scala

// Step 1: display total number of crimes in each year

val displayYear=spark.sql("select Year as Year,count(*) as countOfCrimesInYear from crime_table group by Year order by Year")

// Step 2: Using the lag function to get the variation in crime rates for each year
displayYear.createOrReplaceTempView("year_counts")
val trendresult=spark.sql("""select Year,abs(previouscount-countOfCrimesInYear) as variation_value from(select Year,countOfCrimesInYear,LAG(countOfCrimesInYear,1)  over (order by Year) as previouscount from year_counts) where previouscount is not null""")

// Step 3: finding the years which has minimum and maximum variation
trendresult.createOrReplaceTempView("trendresult")
spark.sql("select Year,variation_value as Minimum_Crime_Trend from trendresult where variation_value=(select min(variation_value) from trendresult)").show
spark.sql("select Year,variation_value as Maximum_Crime_Trend from trendresult where variation_value=(select max(variation_value) from trendresult )").show



+----+-------------------+
Year|Minimum_Crime_Trend|
+----+-------------------+
2006| 0|
+----+-------------------+

+----+-------------------+
Year|Maximum_Crime_Trend|
+----+-------------------+
2019| 162965|
+----+-------------------+

displayYear: org.apache.spark.sql.DataFrame = [Year: int, countOfCrimesInYear: bigint]
trendresult: org.apache.spark.sql.DataFrame = [Year: int, variation_value: bigint]

In [0]:
%scala

spark.sql("select count(*) as Crime_count,`Location Description` from crime_table where `Location Description` is not null group by `Location Description` order by Crime_count desc").show()

+-----------+--------------------+
Crime_count|Location Description|
+-----------+--------------------+
 234983| STREET|
 177593| RESIDENCE|
 134181| APARTMENT|
 87185| SIDEWALK|
 42201| OTHER|
 31593|PARKING LOT/GARAG...|
 26009| SMALL RETAIL STORE|
 25351| RESTAURANT|
 20392|RESIDENTIAL YARD ...|
 19980| ALLEY|
 18583|VEHICLE NON-COMME...|
 18357|RESIDENCE PORCH/H...|
 17696| RESIDENCE-GARAGE|
 17652| DEPARTMENT STORE|
 14313|SCHOOL, PUBLIC, B...|
 13870| GAS STATION|
 13422| GROCERY FOOD STORE|
 7694| BAR OR TAVERN|
 7544| PARK PROPERTY|
 7153| CONVENIENCE STORE|
+-----------+--------------------+
only showing top 20 rows

In [0]:
%scala
// Step 1: Creating an intermediate dataframe by grouping the columns 'Primary Type' and 'Location Description' of the 'crime_table'. It has one more column 'crime_count' representing the number of records in the respective groups i.e; actually the number of occurences of the 'Primary Type' in the respective locations.

val tempCrimeDF=spark.sql("SELECT `Primary Type`, `Location Description`, COUNT(*) AS crime_count FROM crime_table GROUP BY `Primary Type`,`Location Description` ORDER BY `Primary Type`, crime_count DESC")

tempCrimeDF.createOrReplaceTempView("temp_table1")

spark.sql("SELECT t1.`Primary Type`,t1.`Location Description`,t1.crime_count FROM temp_table1 t1 INNER JOIN (SELECT `Primary Type`, MAX(crime_count) AS max_crime_count FROM temp_table1 GROUP BY `Primary Type`) AS t2 ON t1.`Primary Type`=t2.`Primary Type` AND t1.crime_count=t2.max_crime_count ORDER BY crime_count DESC").show()


+--------------------+--------------------+-----------+
 Primary Type|Location Description|crime_count|
+--------------------+--------------------+-----------+
 THEFT| STREET| 58021|
 BATTERY| APARTMENT| 47587|
 CRIMINAL DAMAGE| STREET| 41026|
 MOTOR VEHICLE THEFT| STREET| 31519|
 OTHER OFFENSE| RESIDENCE| 26502|
 DECEPTIVE PRACTICE| RESIDENCE| 23008|
 NARCOTICS| STREET| 17823|
 BURGLARY| RESIDENCE| 16439|
 ROBBERY| SIDEWALK| 13542|
 ASSAULT| STREET| 13260|
 WEAPONS VIOLATION| STREET| 7156|
OFFENSE INVOLVING...| RESIDENCE| 4548|
 CRIMINAL TRESPASS| RESIDENCE| 3359|
INTERFERENCE WITH...| STREET| 2525|
 PROSTITUTION| STREET| 2280|
PUBLIC PEACE VIOL...| STREET| 1800|
 CRIM SEXUAL ASSAULT| RESIDENCE| 1608|
 SEX OFFENSE| RESIDENCE| 1052|
 GAMBLING| SIDEWALK| 557|
 ARSON|VEHICLE NON-COMME...| 349|
+--------------------+--------------------+-----------+
only showing top 20 rows

tempCrimeDF: org.apache.spark.sql.DataFrame = [Primary Type: string, Location Description: string ... 1 more field]

In [0]:
%scala
spark.sql("select `Primary Type`,`Location Description`,count(*) as Crime_count from crime_table where `Location Description` is not null and `Primary Type` in (select `Primary Type` from crime_table group by `Primary Type` having count(*) in (select count(*) as count from crime_table group by `Primary Type` order by count desc limit 5)) group by `Primary Type`,`Location Description` order by `Primary Type`,Crime_count desc ").show()

+------------+--------------------+-----------+
|Primary Type|Location Description|Crime_count|
+------------+--------------------+-----------+
|     ASSAULT|              STREET|      13260|
|     ASSAULT|           RESIDENCE|      12508|
|     ASSAULT|           APARTMENT|      11364|
|     ASSAULT|            SIDEWALK|       8815|
|     ASSAULT|SCHOOL, PUBLIC, B...|       2734|
|     ASSAULT|               OTHER|       2569|
|     ASSAULT|RESIDENCE PORCH/H...|       2333|
|     ASSAULT|          RESTAURANT|       1952|
|     ASSAULT|RESIDENTIAL YARD ...|       1872|
|     ASSAULT|PARKING LOT/GARAG...|       1803|
|     ASSAULT|               ALLEY|       1684|
|     ASSAULT|  SMALL RETAIL STORE|       1457|
|     ASSAULT|         GAS STATION|       1294|
|     ASSAULT|  GROCERY FOOD STORE|        753|
|     ASSAULT|SCHOOL, PUBLIC, G...|        733|
|     ASSAULT|   CONVENIENCE STORE|        693|
|     ASSAULT|       PARK PROPERTY|        531|
|     ASSAULT|    DEPARTMENT STORE|     